In [ ]:
# import libraries
# import matplotlib.pyplot as plt
# import numpy as np
from cil.optimisation.functions import KullbackLeibler, MixedL21Norm
from cil.optimisation.operators import GradientOperator
from cil.optimisation.algorithms import PDHG
from cil.framework import ImageGeometry
from cil.utilities import noise

from cil.utilities.display import show2D
from skimage.metrics import structural_similarity as ssim
import os, sys
import tomophantom
from tomophantom import TomoP2D

In [ ]:
# Load a tomophantom image 
model = 12 # select a model number from the library
N = 256 # set dimension of the phantom
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

phantom2D = TomoP2D.Model(model, N, path_library2D)    
ig = ImageGeometry(voxel_num_x=N, voxel_num_y=N)

In [ ]:
# Fill the phantom 
data = ig.allocate()
data.fill(phantom2D)

In [ ]:
# Corrupt with poisson noise
noisy_data = noise.poisson(data, seed = 10)

In [ ]:
alpha = 0.3

F = alpha * MixedL21Norm()
G = KullbackLeibler(b=noisy_data)
K = GradientOperator(ig)

# Compute operator Norm
normK = K.norm()

# Primal & dual stepsizes
sigma = 1./normK
tau = 1./normK

# Setup and run the PDHG algorithm
pdhg = PDHG(f=F, g=G, operator=K, tau=tau, sigma=sigma,
            max_iteration = 500, update_objective_interval = 100)
pdhg.run(verbose=2)

In [ ]:
val_ssim_tv = ssim(data.as_array(), pdhg.solution.as_array(), data_range = 1.)

In [ ]:
show2D([data, noisy_data, pdhg.solution, (pdhg.solution-data).abs()],
          title=["Ground Truth", 
                  "Corrupted Data (Poisson)", 
                  "Total variation restoration (SSIM = {:.2f}): alpha = {}".format(val_ssim_tv, alpha),
                  "Absolute Difference"],
         origin = "upper", cmap="inferno", size=(10,10))